In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
import transformers
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
import re

2024-03-21 14:09:19.995059: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 14:09:19.995155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 14:09:20.147172: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!pip install git+https://github.com/huggingface/transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_elzrgqb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_elzrgqb
  Resolved https://github.com/huggingface/transformers to commit de627f5a14a777d5cbf21edb78ed67ceb900f8dd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 974.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 14.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 1.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import pandas as pd
from langchain_community.embeddings.openai import OpenAIEmbeddings
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
train = pd.read_csv("/kaggle/input/train-datasets/train_data.csv")
def remove_newlines(df):
    df = df.replace("\n", '', regex=True)
    return df
train = remove_newlines(train)

In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import re

def divide_text_into_batches(text, n_batches):
    """
    Divise un texte en n_batches parties égales.
    """
    batch_size = len(text) // n_batches
    return [text[i:i+batch_size] for i in range(0, len(text), batch_size)]

def extract_sentences_around_dates(batch, model, tokenizer, context_window):
    """
    Extrait les dates et environ 50 mots avant et après ces dates dans un batch de texte.
    """
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)
    results = ner_pipeline(batch)
    dates_and_contexts = []

    for result in results:
        if result['entity'] == 'B-DATE' or result['entity'] == 'I-DATE':
            # Identifier le contexte complet de la date dans le batch
            start, end = result['start'], result['end']
            pre_context = batch[max(0, start - context_window*6):start]  # Estimation approximative
            post_context = batch[end:end + context_window*6]
            
            # Extraire la phrase complète autour de la date
            sentence_boundaries = re.search(r'([.!?]\s+)|([.!?]$)', pre_context[::-1])
            pre_sentence_boundary = -sentence_boundaries.start(0) if sentence_boundaries else -len(pre_context)
            sentence_boundaries = re.search(r'([.!?]\s+)|([.!?]$)', post_context)
            post_sentence_boundary = sentence_boundaries.end(0) if sentence_boundaries else len(post_context)
            
            full_context = pre_context[pre_sentence_boundary:] + batch[start:end] + post_context[:post_sentence_boundary]
            
            # Nettoyer et extraire les mots autour de la date pour limiter à environ 50 mots avant et après
            words_around_date = re.findall(r'\w+', full_context)
            date_index = len(re.findall(r'\w+', pre_context[pre_sentence_boundary:]))
            start_context = max(0, date_index - context_window)
            end_context = min(len(words_around_date), date_index + context_window)
            
            context_sentence = ' '.join(words_around_date[start_context:end_context])
            dates_and_contexts.append((batch[start:end], context_sentence))

    return dates_and_contexts

def extract_dates_with_context_from_long_text(text, model, tokenizer, n_batches, context_window):
    """
    Gère les textes longs par découpage en batches et extrait les phrases autour des dates identifiées.
    """
    batches = divide_text_into_batches(text, n_batches)
    all_dates_and_contexts = []

    for batch in batches:
        batch_dates_and_contexts = extract_sentences_around_dates(batch, model, tokenizer, context_window)
        all_dates_and_contexts.extend(batch_dates_and_contexts)

    return all_dates_and_contexts


model_name = "Jean-Baptiste/camembert-ner-with-dates"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
text = train["texte"][0]
N_BATCHES = 15
CONTEXT = 50

dates_and_contexts = extract_dates_with_context_from_long_text(text, model, tokenizer,
                                                               N_BATCHES, CONTEXT)

config.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

In [9]:
list_context = []
for _, context in dates_and_contexts:
    list_context.append(context)
list_context = list(dict.fromkeys(list_context))

In [10]:
df = pd.DataFrame(list_context, columns=['texte'])
df["filename"] = train.iloc[0]["filename"]
df

,texte,filename
0,Le 12 11 2019 Cour d appel d Agen chambre soci...,Agen_100515.txt
1,GROUPE AZUR ASSURANCES IARD CAISSE PRIMAIRE D ...,Agen_100515.txt
2,AZUR ASSURANCES IARD CAISSE PRIMAIRE D ASSURAN...,Agen_100515.txt
3,SSURANCES IARD CAISSE PRIMAIRE D ASSURANCE MAL...,Agen_100515.txt
4,NCES IARD CAISSE PRIMAIRE D ASSURANCE MALADIE ...,Agen_100515.txt
5,ARD CAISSE PRIMAIRE D ASSURANCE MALADIE DE LOT...,Agen_100515.txt
6,tion judiciaire de la société des transports M...,Agen_100515.txt
7,n judiciaire de la société des transports MORE...,Agen_100515.txt
8,udiciaire de la société des transports MOREL A...,Agen_100515.txt
9,de la société des transports MOREL ARRET N COU...,Agen_100515.txt


In [13]:
# from tqdm import tqdm

# data_to_append = []

# for index, row in tqdm(train.iterrows(), total=train.shape[0]):
#    text = row["texte"]
#    filename = row["filename"]
    
#    dates_and_contexts = extract_dates_with_context_from_long_text(text, model, tokenizer, N_BATCHES, CONTEXT)
    
#    contexts = " ".join([context for _, context in dates_and_contexts])
    
#    data_to_append.append({"filename": filename, "contexts": contexts})

# results_df = pd.DataFrame(data_to_append, columns=["filename", "contexts"])


100%|██████████| 2/2 [00:10<00:00,  5.31s/it]


In [6]:
results_df = pd.read_csv("/kaggle/input/dates-dataset/passages_dates.csv", index_col=[0])

In [7]:
results_df.rename(columns={"contexts": "texte"}, inplace=True)

In [8]:
results_df

,filename,texte
0,Agen_100515.txt,Le 12 11 2019 Cour d appel d Agen chambre soci...
1,Agen_1100752.txt,Le 12 11 2019 Cour d appel d Agen chambre civi...
2,Agen_1613.txt,Le 12 11 2019 Cour d appel d Agen Audience pub...
3,Agen_2118.txt,Le 12 11 2019 Cour d appel d Agen Audience pub...
4,Agen_21229.txt,Le 12 11 2019 Cour d appel d Agen Audience pub...
...,...,...
765,Versailles_602516.txt,Le 12 11 2019 Cour d appel de Versailles ct003...
766,Versailles_605885.txt,Le 12 11 2019 Cour d appel de Versailles ct008...
767,Versailles_61934.txt,Le 12 11 2019 Cour d appel de Versailles ct008...
768,Versailles_67325.txt,Le 12 11 2019 Cour d appel de Versailles ct008...


In [9]:
results_df = results_df[:2]

# RAG

In [12]:
model = transformers.AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", device_map='auto')
tokenizer = transformers.AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [13]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt') # Décommente cette ligne si 'punkt' n'a pas déjà été téléchargé

def preprocess_documents(df_or_series):
    pages_content = []
    
    # Vérifie si l'objet est une Series et le convertit en DataFrame si nécessaire
    if isinstance(df_or_series, pd.Series):
        df = df_or_series.to_frame().transpose()
    else:
        df = df_or_series
    
    for index, row in df.iterrows():
        texte = row["texte"]
        name = row["filename"]
        
        # Découpe le texte en phrases
        phrases = sent_tokenize(texte)
        
        for phrase in phrases:
            texte_doc = Document(page_content=phrase,
                                 metadata={'name': name})
            pages_content.append(texte_doc)
        
    return pages_content

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
content = preprocess_documents(results_df)
embedding_model = SentenceTransformerEmbeddings(model_name='multi-qa-distilbert-cos-v1')
faiss_index = FAISS.from_documents(content, embedding_model)
retriever = faiss_index.as_retriever(search_kwargs={'k': 3})

In [17]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=2000)

In [18]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [19]:
qa = ConversationalRetrievalChain.from_llm(llm=mistral_llm,
                                           retriever=retriever,
                                           return_source_documents=True,
                                           verbose=False)

In [20]:
queries = ["Quelle est la date de l'accident?"]
for query in queries :

    if query == "Quelle est la date de l'accident?":
        result = qa({"question" : f"""Tu es un assistant juridique. Tu vas recevoir plusieurs textes juridiques et tu vas devoir répondre à la recherché une ou des informations demandée(s) par l'utilisateur à partir des documents données.
    On va te demander les informations suivantes : Sexe de la victime, date de l'accident, date de consolidation.
    La date de l'accident correspond à la date à laquelle l'accident s'est produit. Elle est toujours quelque part dans le document (généralement au début) sauf dans de très rares cas.
    voici la demande : {query} ATTENTION: Tu me renverras la date au format MM-DD-AAAA
""",
    "chat_history": []})

    elif query == "Quelle est la date de consolidation?":
        result = qa({"question" : f"""Tu es un assistant juridique. Tu vas recevoir plusieurs textes juridiques et tu vas devoir répondre à la recherché une ou des informations demandée(s) par l'utilisateur à partir des documents données.
    On va te demander les informations suivantes : Sexe de la victime, date de l'accident, date de consolidation.
    La date de consolidation est la date à laquelle les blessures de la victime sont devenues stables et ont été déclarées définitives par un médecin. L'information devrait être présente dans la plupart des cas mais parfois elle est soit manquante (tu afficheras la valeur "n.c.") soit non applicable (tu afficheras la valeur "n.a.") si la blessure n'a pas stabilisé avant le décès de la victime.
    voici la demande : {query} """,
    "chat_history": []})


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  """
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_34/453977644.py", line 5, in <module>
    result = qa({"question" : f"""Tu es un assistant juridique. Tu vas recevoir plusieurs textes juridiques et tu vas devoir répondre à la recherché une ou des informations demandée(s) par l'utilisateur à partir des documents données.
  File "/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py", line 145, in warning_emitting_wrapper
    return wrapped(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 378, in __call__
    return self.invoke(
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_

In [ ]:
for element in result["source_documents"]:
    source = f'\n- {element.metadata["name"]}'
    print(source)

In [23]:
print(result['answer'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

avisées de la date à laquelle l arrêt serait rendu Le 9 avril 1991 Monsieur X a été victime d un accident du travail au cours duquel il a été gravement blessé Suivant jugement en date du 24 avril 1997 confirmé par arrêt de la Cour en date du 30 mars 1999 le Tribunal des Affaires de Sé

été avisées de la date à laquelle l arrêt serait rendu Le 9 avril 1991 Monsieur X a été victime d un accident du travail au cours duquel il a été gravement blessé Suivant jugement en date du 24 avril 1997 confirmé par arrêt de la Cour en date du 30 mars 1999 le Tribunal des Affaires de Sé

de la date à laquelle l arrêt serait rendu Le 9 avril 1991 Monsieur X a été victime d un accident du travail au cours duquel il a été gravement blessé Suivant jugement en date du 24 avril 1997 confirmé par arrêt de la Cour en date du 30 mars 1999 le Tribunal